In [1]:
import json
import pandas as pd
from tqdm import tqdm_notebook
from IPython.display import HTML, display
import tabulate
import sqlite3
from sqlite3 import Error
import random
import copy
import math


human_labels = ('woman', 'men', 'boy', 'kid', 'child', 'guy', 'man', 'person', 'girl', 'lady', 'people')

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [3]:
con = sqlite3.connect('person.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('images',), ('objects',), ('relations',)]


In [4]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('Eurocity',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_eurocity_df = pd.DataFrame(table, columns =head) 
# r_eurocity_df

In [6]:
r_eurocity_df['images'] = ""
# import json
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()

# rel_dict = dict()

# for index, row in r_eurocity_df.iterrows():
for index, row in tqdm_notebook(r_eurocity_df.iterrows()):
#     print(f"{r_eurocity_df.at[index, 'Object_1']}, {r_eurocity_df.at[index, 'Relation']}, {r_eurocity_df.at[index, 'Object_2']}")
    q_in = ('Eurocity' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
#         im_set.add(r)
    r_eurocity_df.at[index, 'images'] = list(t_st)

r_eurocity_df

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images
0,building,behind,man,13315,"[zuerich_00030, budapest_00163, nuernberg_0038..."
1,building,behind,person,11261,"[zuerich_00030, budapest_00163, stuttgart_0011..."
2,person,walking on,street,8611,"[firenze_01074, stuttgart_00111, nuernberg_003..."
3,person,walking on,sidewalk,7407,"[amsterdam_00743, budapest_00151, milano_00728..."
4,man,walking on,street,6689,"[prague_00585, budapest_00151, firenze_01109, ..."
...,...,...,...,...,...
677,woman,watching,boy,1,[firenze_00429]
678,woman,watching,dog,1,[milano_00406]
679,woman,watching,light,1,[budapest_00267]
680,woman,watching,umbrella,1,[nuernberg_00039]


In [7]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('Caltech',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_caltech_df = pd.DataFrame(table, columns =head) 

In [8]:
r_caltech_df['images'] = ""
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()

rel_dict = dict()
for index, row in r_caltech_df.iterrows():
    q_in = ('Caltech' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=?", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
    r_caltech_df.at[index, 'images'] = list(t_st)

r_caltech_df

,Object_1,Relation,Object_2,Count,images
0,person,walking on,street,618,"[set03_V004_914, set05_V002_1582, set00_V009_1..."
1,person,walking on,sidewalk,468,"[set03_V003_235, set05_V002_1582, set00_V009_1..."
2,man,walking on,street,421,"[set03_V003_235, set05_V002_1582, set01_V000_1..."
3,building,behind,person,337,"[set03_V003_235, set05_V002_1582, set00_V009_1..."
4,building,behind,man,336,"[set03_V003_235, set05_V002_1582, set01_V000_1..."
...,...,...,...,...,...
214,woman,holding,pole,1,[set05_V012_673]
215,woman,sitting on,chair,1,"[set05_V011_610, set01_V004_1104, set00_V012_900]"
216,woman,watching,car,1,[set00_V008_1476]
217,woman,watching,letter,1,"[set00_V007_1275, set00_V008_1476]"


In [9]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('CityPerson',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_cityperson_df = pd.DataFrame(table, columns =head) 
r_cityperson_df 

,Object_1,Relation,Object_2,Count
0,building,behind,man,1746
1,building,behind,person,1239
2,person,walking on,sidewalk,1071
3,person,walking on,street,909
4,tree,behind,man,889
...,...,...,...,...
259,woman,looking at,tree,1
260,woman,riding,motorcycle,1
261,woman,sitting on,bench,1
262,woman,sitting on,chair,1


In [10]:

r_cityperson_df['images'] = ""
database = r"person.db"
im_set = set()
# create a database connection
conn = create_connection(database)
cur = conn.cursor()
for index, row in tqdm_notebook(r_cityperson_df.iterrows(), desc ="update the r_cityperson_df"):
    q_in = ('CityPerson' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
        im_set.add(r[0])
#         for el in r:
#             im_set.add(el)
    r_cityperson_df.at[index, 'images'] = list(t_st)

r_cityperson_df
# len(im_set)

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images
0,building,behind,man,1746,"[hamburg_000000_052122_leftImg8bit, darmstadt_..."
1,building,behind,person,1239,"[hamburg_000000_052122_leftImg8bit, hamburg_00..."
2,person,walking on,sidewalk,1071,"[hamburg_000000_061468_leftImg8bit, ulm_000063..."
3,person,walking on,street,909,"[ulm_000063_000019_leftImg8bit, jena_000036_00..."
4,tree,behind,man,889,"[hamburg_000000_052122_leftImg8bit, darmstadt_..."
...,...,...,...,...,...
259,woman,looking at,tree,1,[bochum_000000_009554_leftImg8bit]
260,woman,riding,motorcycle,1,[dusseldorf_000165_000019_leftImg8bit]
261,woman,sitting on,bench,1,[hamburg_000000_098061_leftImg8bit]
262,woman,sitting on,chair,1,[hamburg_000000_052122_leftImg8bit]


In [11]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('CrowdHuman',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_crowdhuman_df = pd.DataFrame(table, columns =head) 
# crowdhuman_df 

In [12]:
r_crowdhuman_df['images'] = ""
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()
for index, row in tqdm_notebook(r_crowdhuman_df.iterrows(), desc ="update the r_cityperson_df"):
    q_in = ('CrowdHuman' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=?", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
    r_crowdhuman_df.at[index, 'images'] = list(t_st)

r_crowdhuman_df

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images
0,man,watching,man,12330,"[273275,10f1b60006d757b58, 282555,1247f0002190..."
1,building,behind,man,8428,"[283647,f2e20005ddc2d0f, 282555,d912e00083b9a7..."
2,person,watching,man,7706,"[283647,1ab170009ad95850, 282555,da567000944cf..."
3,building,behind,woman,7382,"[282555,d912e00083b9a7d1, 273275,c4eac00002dcf..."
4,man,wearing,shirt,6885,"[273275,10eced000cead4fd8, 273278,114ec000a7b0..."
...,...,...,...,...,...
834,woman,watching,people,1,"[282555,a15b4000b3a70c1c, 282555,b4e970002562f..."
835,woman,watching,sheep,1,"[273275,3e5f000747f9295, 282555,a64bd000f51e539d]"
836,woman,watching,tail,1,"[273278,34dfc00090520457]"
837,woman,wearing,logo,1,"[273271,69a80006c4e599f]"


In [13]:
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()


cur.execute("SELECT object_1,relation,object_2, count(*) FROM relations WHERE dataset_name=? and object1_score > 0.15 and object2_score > 0.15 and relation_score > 0.20 GROUP BY object_1,relation,object_2  ORDER BY count(*) DESC", ('WiderPerson',))

rows = cur.fetchall()

head = ["Object_1", "Relation", "Object_2", "Count"]
table = []

for row in rows:
    table.append(row)

r_widerperson_df = pd.DataFrame(table, columns=head) 
r_widerperson_df 

,Object_1,Relation,Object_2,Count
0,man,watching,man,8605
1,person,watching,man,8527
2,person,watching,woman,5479
3,building,behind,man,5058
4,building,behind,woman,4100
...,...,...,...,...
508,woman,in front of,door,1
509,woman,under,umbrella,1
510,woman,watching,animal,1
511,woman,watching,cow,1


In [14]:
r_widerperson_df['images'] = ""
database = r"person.db"

# create a database connection
conn = create_connection(database)
cur = conn.cursor()
for index, row in tqdm_notebook(r_widerperson_df.iterrows(), desc ="update the r_cityperson_df"):
    q_in = ('WiderPerson' ,row['Object_1'], row['Relation'], row['Object_2'])
    cur.execute("SELECT image_name FROM relations WHERE dataset_name=? and object_1=? and relation=? and object_2=?", q_in)
    rows = cur.fetchall()
    t_st = set()
    for r in rows:
        t_st.add(r[0])
    r_widerperson_df.at[index, 'images'] = list(t_st)

r_widerperson_df

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images
0,man,watching,man,8605,"[001147, 000488, 015248, 003998, 018494, 01043..."
1,person,watching,man,8527,"[001147, 000488, 013211, 015248, 001988, 00399..."
2,person,watching,woman,5479,"[001988, 003066, 018494, 000426, 005913, 01621..."
3,building,behind,man,5058,"[001147, 000488, 013211, 004407, 003998, 01849..."
4,building,behind,woman,4100,"[004407, 018494, 000426, 016215, 001233, 01674..."
...,...,...,...,...,...
508,woman,in front of,door,1,"[009452, 016526]"
509,woman,under,umbrella,1,[010318]
510,woman,watching,animal,1,[016979]
511,woman,watching,cow,1,[013810]


In [15]:
def setA_setB(A, B):
    """
    set A - set B
    """
    common_list = []
    diff_set = []
    comm_set = []
    for idx, row in A.iterrows():
        index_list = B[(B['Object_1'] == row['Object_1'])&(B['Relation'] == row['Relation'])&(B['Object_2'] == row['Object_2'])].index.tolist()
        if not index_list:
           diff_set.append(idx) 
        else:
           comm_set.append(idx)
    diff_cp = A.iloc[diff_set]
    com_cp = A.iloc[comm_set]
    return diff_cp, com_cp

def find_quantile(diff_set,q=0.95, max_im=20):
    quantile_95 = diff_set[diff_set.Count > diff_set.Count.quantile(q)]
    quantile_95 = quantile_95.sort_values('Count', ascending=False)
    quantile_95.reset_index(drop=True, inplace=True)
    quantile_95['count_down'] = max_im
#     im_to_add = []
    for index, row in tqdm_notebook(quantile_95.iterrows(), desc ="update the r_cityperson_df"):
        if len(quantile_95.at[index, 'images']) > quantile_95.at[index, 'count_down']:
            s_images = random.sample(quantile_95.at[index, 'images'], quantile_95.at[index, 'count_down'])
            quantile_95.at[index, 'images'] = s_images
        else:
            s_images = quantile_95.at[index, 'images']
        quantile_95.at[index, 'count_down'] = quantile_95.at[index, 'count_down'] - len(s_images)
#         im_to_add.extend(s_images)

#     im_to_add = list(set(im_to_add))
#     return im_to_add, quantile_95
    return quantile_95

def splite_data(data_df, train_size=0.55, val_size=0.1, test_size=0.35):
    data_df = copy.deepcopy(data_df)
    all_images = list(set([item for sublist in data_df['images'].tolist() for item in sublist]))
    train_images = random.sample(all_images, int(len(all_images) * train_size))
    rem_images = list(set(all_images).difference(set(train_images)))
    val_images = random.sample(rem_images, int(math.ceil(len(all_images) * val_size)))
    test_images = list(set(all_images).difference(set(train_images).union(set(val_images))))
    
#     print(set(train_images).intersection(set(val_images)))

    assert len(all_images) == len(train_images) + len(val_images) + len(test_images)
    assert len(set(train_images).intersection(set(val_images))) == 0
    assert len(set(train_images).intersection(set(test_images))) == 0
    assert len(set(val_images).intersection(set(test_images))) == 0
    return train_images, val_images, test_images

def concat_df(df_list):
    bigdata = copy.deepcopy(df_list[0])
    for el in df_list[1:]:
        bigdata = bigdata.append(el, ignore_index=True)
    return bigdata

def find_index_of_images(img_df, image_name_list=None):
    index_id_list = []
    for el in image_name_list:
        index_ids = img_df.index[img_df['file_name'].str.contains(el)].tolist()
        image_id = [img_df.at[idx, 'id'] for idx in index_ids]
        if len(image_id) != 1:
            print(el, image_id)
        assert len(image_id) == 1
        index_id_list.append(index_ids[0])
    return index_id_list

def create_ann_df(ann_df, index_list):
    selected_ann_df = ann_df[ann_df['image_id'].isin([index_list[0] + 1])]
    for el in index_list[1:]:
        selected_ann_df = pd.concat([selected_ann_df,ann_df[ann_df['image_id'].isin([el + 1])]])
    return selected_ann_df

def create_img_df(img_df, index_list):
    selected_img_df = img_df.loc[[index_list[0]]]
    for el in index_list[1:]:
        selected_img_df = pd.concat([selected_img_df,img_df.loc[[el]]])
    return selected_img_df

def update_img_ids(aug_img_df, row=None, dataset='other'):
    last_id = aug_img_df.at[aug_img_df.index[-1], 'id']
    concat_df = pd.concat([aug_img_df, row])
    concat_df.reset_index(drop=True, inplace=True)
    concat_df.at[concat_df.index[-1], 'id'] = last_id + 1
    return last_id + 1, concat_df

def uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df, dt='other'):
    selected_img_df = selected_img_df.sort_values('id')
    if dt == 'ch':
        selected_ann_df['height'] = ""
        selected_ann_df['width'] = ""
        for idx, row in selected_img_df.iterrows():
            new_image_id, aug_img_df = update_img_ids(aug_img_df, row=selected_img_df.loc[[idx]], dataset='ch')
            
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'height'] = aug_img_df.at[new_image_id -1, 'height']
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'width'] = aug_img_df.at[new_image_id -1, 'width']
            
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'image_id'] = new_image_id

    else:
        for idx, row in selected_img_df.iterrows():
            new_image_id, aug_img_df = update_img_ids(aug_img_df, row=selected_img_df.loc[[idx]])
            selected_ann_df.loc[selected_ann_df['image_id'] == idx + 1, 'image_id'] = new_image_id
    
    return aug_img_df, selected_ann_df

def concat_ann_df(orig_df, add_df):
    ann_df_c = pd.concat([orig_df, add_df])
    ann_df_c.reset_index(drop=True, inplace=True)
    for idx, row in ann_df_c.iterrows():
        if idx + 1 != row['id']:
            ann_df_c.at[idx, 'id'] = idx + 1
    return ann_df_c

## Find diff between the relation datasets

In [16]:
clt_cp, _ = setA_setB(r_caltech_df, r_cityperson_df)
clt_cp

,Object_1,Relation,Object_2,Count,images
49,person,watching,letter,11,"[set00_V013_1399, set00_V013_1249, set00_V007_..."
53,mountain,behind,person,9,"[set03_V003_55, set05_V003_671, set03_V003_235..."
54,people,watching,letter,9,"[set00_V013_1399, set00_V013_1249, set00_V007_..."
62,person,watching,truck,6,"[set00_V010_81, set01_V001_1349, set00_V004_14..."
70,light,above,person,4,"[set00_V008_1506, set00_V010_81, set00_V008_13..."
...,...,...,...,...,...
213,woman,holding,handle,1,[set00_V006_625]
214,woman,holding,pole,1,[set05_V012_673]
216,woman,watching,car,1,[set00_V008_1476]
217,woman,watching,letter,1,"[set00_V007_1275, set00_V008_1476]"


In [17]:
clt_cp, _ = setA_setB(r_caltech_df, r_cityperson_df)
clt_cp

,Object_1,Relation,Object_2,Count,images
49,person,watching,letter,11,"[set00_V013_1399, set00_V013_1249, set00_V007_..."
53,mountain,behind,person,9,"[set03_V003_55, set05_V003_671, set03_V003_235..."
54,people,watching,letter,9,"[set00_V013_1399, set00_V013_1249, set00_V007_..."
62,person,watching,truck,6,"[set00_V010_81, set01_V001_1349, set00_V004_14..."
70,light,above,person,4,"[set00_V008_1506, set00_V010_81, set00_V008_13..."
...,...,...,...,...,...
213,woman,holding,handle,1,[set00_V006_625]
214,woman,holding,pole,1,[set05_V012_673]
216,woman,watching,car,1,[set00_V008_1476]
217,woman,watching,letter,1,"[set00_V007_1275, set00_V008_1476]"


In [18]:
all_images = list(set([item for sublist in clt_cp['images'].tolist() for item in sublist]))
len(all_images)

366

In [19]:
all_images = list(set([item + '.png' for sublist in clt_cp['images'].tolist() for item in sublist]))
all_images

['set01_V000_780.png',
 'set02_V008_1236.png',
 'set00_V012_1110.png',
 'set02_V003_237.png',
 'set00_V012_540.png',
 'set04_V005_78.png',
 'set05_V011_1450.png',
 'set00_V009_1423.png',
 'set00_V013_1399.png',
 'set02_V009_896.png',
 'set00_V009_133.png',
 'set00_V006_535.png',
 'set00_V013_559.png',
 'set00_V007_1275.png',
 'set01_V002_677.png',
 'set00_V009_643.png',
 'set00_V009_1063.png',
 'set03_V004_794.png',
 'set01_V001_1379.png',
 'set00_V008_1356.png',
 'set04_V002_847.png',
 'set05_V005_499.png',
 'set00_V010_231.png',
 'set00_V013_109.png',
 'set00_V013_19.png',
 'set04_V008_1059.png',
 'set00_V007_345.png',
 'set00_V004_811.png',
 'set04_V007_1671.png',
 'set00_V010_261.png',
 'set02_V008_1476.png',
 'set02_V009_56.png',
 'set03_V010_1085.png',
 'set00_V001_1275.png',
 'set04_V005_1668.png',
 'set00_V014_488.png',
 'set03_V012_100.png',
 'set00_V008_1176.png',
 'set01_V001_1169.png',
 'set00_V003_479.png',
 'set00_V008_36.png',
 'set00_V013_379.png',
 'set01_V004_1434.png

In [20]:
train_img_clt, val_img_clt, test_img_clt = splite_data(clt_cp, train_size=0.7, val_size=0.15, test_size=0.15)
ee = [el + '.png' for el in train_img_clt]
ee

['set02_V008_1566.png',
 'set00_V006_865.png',
 'set04_V001_1668.png',
 'set00_V007_255.png',
 'set00_V007_1785.png',
 'set00_V010_1041.png',
 'set00_V008_906.png',
 'set02_V010_674.png',
 'set02_V008_1446.png',
 'set00_V013_49.png',
 'set00_V013_1039.png',
 'set00_V012_1320.png',
 'set05_V011_850.png',
 'set02_V009_146.png',
 'set00_V001_1125.png',
 'set00_V014_308.png',
 'set03_V003_145.png',
 'set00_V014_1028.png',
 'set00_V006_265.png',
 'set00_V008_366.png',
 'set05_V005_469.png',
 'set00_V008_1026.png',
 'set00_V008_1386.png',
 'set05_V001_153.png',
 'set00_V002_151.png',
 'set00_V006_745.png',
 'set04_V000_690.png',
 'set00_V010_171.png',
 'set00_V007_675.png',
 'set00_V001_615.png',
 'set00_V002_361.png',
 'set05_V003_671.png',
 'set02_V009_1106.png',
 'set01_V002_677.png',
 'set03_V008_1349.png',
 'set04_V001_1638.png',
 'set00_V007_1575.png',
 'set01_V005_940.png',
 'set01_V001_1349.png',
 'set02_V008_1326.png',
 'set00_V013_1219.png',
 'set03_V009_17.png',
 'set00_V014_1058.

In [21]:
ee = [el + '.png' for el in val_img_clt]
ee

['set00_V006_355.png',
 'set00_V007_1485.png',
 'set00_V012_1080.png',
 'set02_V003_117.png',
 'set00_V012_360.png',
 'set02_V008_1536.png',
 'set02_V003_237.png',
 'set00_V013_1489.png',
 'set00_V007_1395.png',
 'set00_V000_240.png',
 'set03_V003_25.png',
 'set03_V012_40.png',
 'set00_V010_81.png',
 'set04_V003_235.png',
 'set02_V009_686.png',
 'set05_V004_750.png',
 'set02_V009_1616.png',
 'set02_V001_168.png',
 'set04_V002_847.png',
 'set00_V013_109.png',
 'set00_V009_1513.png',
 'set00_V007_1215.png',
 'set00_V014_938.png',
 'set00_V008_216.png',
 'set00_V008_936.png',
 'set03_V003_535.png',
 'set02_V008_1686.png',
 'set00_V003_119.png',
 'set04_V007_1671.png',
 'set01_V000_780.png',
 'set00_V000_870.png',
 'set05_V012_1003.png',
 'set00_V014_1328.png',
 'set02_V009_1166.png',
 'set00_V000_1470.png',
 'set00_V007_1845.png',
 'set03_V003_115.png',
 'set00_V008_1356.png',
 'set00_V007_705.png',
 'set02_V003_147.png',
 'set00_V014_488.png',
 'set02_V010_524.png',
 'set05_V000_120.png'

In [22]:
ee = [el + '.png' for el in test_img_clt]
ee

['set00_V013_1399.png',
 'set00_V007_345.png',
 'set00_V014_1598.png',
 'set00_V014_188.png',
 'set01_V001_1229.png',
 'set00_V001_225.png',
 'set00_V010_891.png',
 'set01_V004_1644.png',
 'set00_V006_955.png',
 'set00_V006_565.png',
 'set00_V012_210.png',
 'set03_V012_100.png',
 'set00_V004_481.png',
 'set03_V008_1259.png',
 'set02_V008_1596.png',
 'set00_V008_1236.png',
 'set00_V002_1021.png',
 'set00_V009_43.png',
 'set02_V011_1682.png',
 'set00_V012_570.png',
 'set00_V002_541.png',
 'set00_V013_739.png',
 'set00_V009_823.png',
 'set00_V000_1530.png',
 'set03_V003_55.png',
 'set00_V008_1176.png',
 'set05_V012_1033.png',
 'set00_V013_1429.png',
 'set00_V008_846.png',
 'set02_V009_1196.png',
 'set00_V013_1099.png',
 'set02_V008_1296.png',
 'set00_V001_45.png',
 'set02_V009_56.png',
 'set02_V004_352.png',
 'set00_V011_809.png',
 'set00_V013_439.png',
 'set04_V001_78.png',
 'set00_V013_379.png',
 'set00_V001_945.png',
 'set05_V007_1347.png',
 'set00_V001_1785.png',
 'set02_V009_866.png'

In [23]:
clt_quantile_95 = find_quantile(clt_cp,q=0.95, max_im=20)
clt_quantile_95

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images,count_down
0,person,watching,letter,11,"[set00_V013_559, set00_V008_846, set00_V014_18...",0
1,mountain,behind,person,9,"[set03_V003_55, set05_V003_671, set03_V003_235...",3
2,people,watching,letter,9,"[set00_V007_1845, set00_V008_1446, set00_V014_...",0
3,person,watching,truck,6,"[set00_V010_1491, set01_V001_1199, set04_V011_...",0


In [24]:
all_images = list(set([item for sublist in clt_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

70

In [25]:
ecp_cp, _ = setA_setB(r_eurocity_df, r_cityperson_df)
ecp_cp

,Object_1,Relation,Object_2,Count,images
84,people,riding,motorcycle,60,"[firenze_01098, bologna_00036, roma_00401, bol..."
85,person,watching,motorcycle,60,"[firenze_01044, barcelona_01025, bologna_00726..."
87,person,wearing,helmet,55,"[roma_00105, firenze_00576, berlin_00096, mila..."
94,sign,above,man,47,"[hamburg_00376, toulouse_00112, budapest_00210..."
103,person,in,window,41,"[amsterdam_00860, ljubljana_00473, amsterdam_0..."
...,...,...,...,...,...
677,woman,watching,boy,1,[firenze_00429]
678,woman,watching,dog,1,[milano_00406]
679,woman,watching,light,1,[budapest_00267]
680,woman,watching,umbrella,1,[nuernberg_00039]


In [26]:
all_images = list(set([item for sublist in ecp_cp['images'].tolist() for item in sublist]))
len(all_images)

1194

In [27]:
ecp_quantile_95 = find_quantile(ecp_cp,q=0.95, max_im=40)
ecp_quantile_95

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images,count_down
0,people,riding,motorcycle,60,"[roma_00401, firenze_00064, wuerzburg_00656, r...",0
1,person,watching,motorcycle,60,"[roma_00773, bologna_00397, basel_00117, bolog...",0
2,person,wearing,helmet,55,"[roma_00411, berlin_00096, milano_00302, roma_...",0
3,sign,above,man,47,"[hamburg_00376, toulouse_00112, budapest_00210...",0
4,person,in,window,41,"[amsterdam_00860, ljubljana_00473, amsterdam_0...",22
5,window,above,man,38,"[wuerzburg_00198, marseille_00304, wuerzburg_0...",9
6,car,near,man,36,"[firenze_00484, zagreb_00078, budapest_00210, ...",8
7,window,above,person,35,"[zagreb_00566, hamburg_00132, dresden_00404, b...",13
8,person,wearing,hat,33,"[ljubljana_00445, amsterdam_00591, amsterdam_0...",15
9,person,watching,truck,32,"[montpellier_00299, roma_00212, leipzig_00025,...",10


In [28]:
all_images = list(set([item for sublist in ecp_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

521

In [29]:
ch_cp, _ = setA_setB(r_crowdhuman_df, r_cityperson_df)
ch_cp

,Object_1,Relation,Object_2,Count,images
32,man,wearing,hat,1691,"[283991,640b00005765892, 273275,bfffc0004e1c62..."
52,woman,watching,girl,827,"[283992,56a000c39f777e, 282555,d030d000c312f11..."
54,woman,watching,boy,771,"[283554,c2740004f2498e1, 282555,d030d000c312f1..."
55,man,wearing,sock,769,"[283647,6b9b000037a02c2, 273271,1a51000317893a..."
60,woman,wearing,boot,673,"[273275,a002b000d796b415, 283992,56a000c39f777..."
...,...,...,...,...,...
834,woman,watching,people,1,"[282555,a15b4000b3a70c1c, 282555,b4e970002562f..."
835,woman,watching,sheep,1,"[273275,3e5f000747f9295, 282555,a64bd000f51e539d]"
836,woman,watching,tail,1,"[273278,34dfc00090520457]"
837,woman,wearing,logo,1,"[273271,69a80006c4e599f]"


In [30]:
all_images = list(set([item for sublist in ch_cp['images'].tolist() for item in sublist]))
len(all_images)

8205

In [31]:
ch_quantile_95 = find_quantile(ch_cp,q=0.95, max_im=20)
ch_quantile_95

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images,count_down
0,man,wearing,hat,1691,"[283554,3a7110003dc5b0bf, 273275,f1d570009b6f0...",0
1,woman,watching,girl,827,"[282555,418f4000e73c080d, 283554,2a868000b5db0...",0
2,woman,watching,boy,771,"[273278,97eb10001f832ccf, 283554,1bdd9000f9b0b...",0
3,man,wearing,sock,769,"[282555,df02000176315ab, 284193,122a9000dc4163...",0
4,woman,wearing,boot,673,"[283081,40300051c350e8, 282555,edf03000899b17a...",0
5,woman,wearing,glass,654,"[273275,7913500026315b86, 273275,83a49000896db...",0
6,person,watching,girl,618,"[284193,15fd5000cfc28f1a, 283554,3bf0e0005df52...",0
7,man,wearing,glass,560,"[273275,ec28c000abccdd87, 273271,170730007f64c...",0
8,people,wearing,short,526,"[273271,34d50002f5cb3ce, 273271,1bc75000ad1a58...",0
9,person,wearing,hat,511,"[284193,1a2240009038ae6c, 283081,1ffc500002af2...",0


In [32]:
all_images = list(set([item for sublist in ch_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

624

In [33]:
wp_cp, _ = setA_setB(r_widerperson_df, r_cityperson_df)
wp_cp

,Object_1,Relation,Object_2,Count,images
19,person,watching,girl,1768,"[001306, 019187, 012035, 003066, 018494, 00079..."
28,woman,watching,girl,1166,"[011051, 003066, 006898, 013115, 014624, 01607..."
36,woman,watching,boy,926,"[001306, 013081, 018982, 007124, 004407, 00451..."
48,people,wearing,short,582,"[006140, 004407, 018663, 014033, 013119, 00326..."
50,mountain,behind,man,522,"[019211, 019312, 012275, 013797, 012055, 00783..."
...,...,...,...,...,...
507,woman,holding,racket,1,"[019273, 012137, 006443, 004716]"
509,woman,under,umbrella,1,[010318]
510,woman,watching,animal,1,[016979]
511,woman,watching,cow,1,[013810]


In [34]:
all_images = list(set([item for sublist in wp_cp['images'].tolist() for item in sublist]))
len(all_images)

4286

In [35]:
wp_quantile_95 = find_quantile(wp_cp,q=0.95, max_im=30)
wp_quantile_95

/home/test_pc/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


update the r_cityperson_df: 0it [00:00, ?it/s]

,Object_1,Relation,Object_2,Count,images,count_down
0,person,watching,girl,1768,"[005957, 004134, 018465, 019361, 001520, 00642...",0
1,woman,watching,girl,1166,"[000367, 005930, 015800, 005597, 009542, 00691...",0
2,woman,watching,boy,926,"[003794, 014442, 011839, 019587, 012889, 01308...",0
3,people,wearing,short,582,"[010311, 013500, 009130, 005245, 005071, 00118...",0
4,mountain,behind,man,522,"[006272, 007412, 016218, 019083, 004795, 00400...",0
5,boy,watching,man,460,"[019064, 008736, 006120, 003055, 009754, 00320...",0
6,mountain,behind,person,434,"[015064, 009516, 004487, 009778, 016874, 01912...",0
7,man,wearing,hat,395,"[005108, 001889, 001273, 016077, 005981, 01524...",0
8,boy,watching,boy,388,"[015853, 000830, 013266, 014116, 008101, 00921...",0
9,girl,watching,woman,330,"[007007, 009349, 000470, 006528, 010690, 00996...",0


In [36]:
all_images = list(set([item for sublist in wp_quantile_95['images'].tolist() for item in sublist]))
len(all_images)

502

In [37]:
df_list = []
df_list.append(ecp_quantile_95)

In [38]:
ch_q_ecp_q, _ = setA_setB(ch_quantile_95, ecp_quantile_95)
# ch_seclected_df = splite_data(ch_q_ecp_q, train_size=0.55, val_size=0.1, test_size=0.35)
# ch_seclected_df

In [39]:
df_list.append(ch_q_ecp_q)

In [40]:
concat_data = concat_df(df_list)
# concat_data

In [41]:
wp_q_con, _ = setA_setB(wp_quantile_95, concat_data)
# wp_seclected_df = splite_data(wp_q_con, train_size=0.55, val_size=0.1, test_size=0.35)
# wp_seclected_df

In [42]:
df_list.append(wp_q_con)

In [43]:
concat_data = concat_df(df_list)
# concat_data

In [44]:
# clt_quantile_95
clt_q_con, _ = setA_setB(clt_quantile_95, concat_data)
# clt_seclected_df = splite_data(clt_q_con, train_size=0.55, val_size=0.1, test_size=0.35)
# clt_seclected_df

In [45]:
df_list.append(clt_q_con)

In [46]:
concat_data = concat_df(df_list)
concat_data

,Object_1,Relation,Object_2,Count,images,count_down
0,people,riding,motorcycle,60,"[roma_00401, firenze_00064, wuerzburg_00656, r...",0
1,person,watching,motorcycle,60,"[roma_00773, bologna_00397, basel_00117, bolog...",0
2,person,wearing,helmet,55,"[roma_00411, berlin_00096, milano_00302, roma_...",0
3,sign,above,man,47,"[hamburg_00376, toulouse_00112, budapest_00210...",0
4,person,in,window,41,"[amsterdam_00860, ljubljana_00473, amsterdam_0...",22
...,...,...,...,...,...,...
56,girl,watching,woman,330,"[007007, 009349, 000470, 006528, 010690, 00996...",0
57,people,watching,girl,268,"[019366, 018578, 007007, 014503, 018329, 00569...",0
58,girl,watching,girl,247,"[012188, 011050, 011848, 016239, 014574, 00966...",0
59,person,watching,letter,11,"[set00_V013_559, set00_V008_846, set00_V014_18...",0


In [47]:
ecp_seclected_tupel = splite_data(df_list[0], train_size=0.55, val_size=0.1, test_size=0.35)
len(ecp_seclected_tupel[0])

286

In [48]:
orig_human_im_list = list(set([item for sublist in r_cityperson_df['images'].tolist() for item in sublist]))
len(set(orig_human_im_list))

1745

In [49]:
def add_area(df):
    df = copy.deepcopy(df)
    df['area'] = ""
    for idx, row in df.iterrows():
        bb = df.loc[df['id'] == idx + 1, 'bbox']
        bb = bb.tolist()[0]
        df.loc[df['id'] == idx + 1, 'area'] = bb[2]*bb[3]
    return df

In [50]:
def splite_dataset(data_df, train_size=50, test_size=10, dt=None):
    if not dt:
        all_images = [el.rsplit('/', 1)[1] for el in data_df['file_name'].to_list()]
    else:
        all_images = data_df['file_name'].to_list()
    train_images = random.sample(all_images, train_size)
    rem_images = list(set(all_images).difference(set(train_images)))
    test_images = random.sample(rem_images, test_size)
    assert len(set(train_images).intersection(set(test_images))) == 0
    return train_images, test_images

In [51]:
def create_test_dfs(in_img_df, in_ann_df, image_name_list):
    index_id_list = find_index_of_images(in_img_df, image_name_list=image_name_list)
    _img_df = create_img_df(in_img_df, index_id_list)
    _ann_df = create_ann_df(in_ann_df, index_id_list)
    _img_df = _img_df.sort_values('id')
    _img_df.reset_index(drop=True, inplace=True)
    
    for idx, row in _img_df.iterrows():
        img_id = _img_df.at[_img_df.index[idx], 'id']
        _img_df.at[_img_df.index[idx], 'id'] = idx + 1
        print(f"try replace {img_id} to {idx + 1} in anno")
        if 'image_id' in _ann_df.columns:
            _ann_df.loc[_ann_df['image_id'] == img_id, 'image_id'] = idx + 1
    
    _ann_df = _ann_df.sort_values('image_id')
    _ann_df.reset_index(drop=True, inplace=True)
    for idx, row in _ann_df.iterrows():
        _ann_df.at[_ann_df.index[idx], 'id'] = idx + 1
    return _img_df, _ann_df

In [52]:
def _ugment_df(in_img_df, in_ann_df, test_img_list, _aug_img_df, _ann_df_c):   
    
    index_id_list = find_index_of_images(in_img_df, image_name_list=test_img_list)
    s_ecp_img_df = create_img_df(in_img_df, index_id_list)
    print(f"you will add {s_ecp_img_df.shape[0]} to images")
    
    s_ecp_ann_df = create_ann_df(in_ann_df, index_id_list)
    print(f"you will add {s_ecp_ann_df.shape[0]} to anno")

    _aug_img_df, s_ecp_ann_df = uodate_id_in_image_ann(s_ecp_img_df, _aug_img_df, s_ecp_ann_df)
    s_ecp_ann_df = add_area(s_ecp_ann_df)

    _ann_df_c = concat_ann_df(_ann_df_c, s_ecp_ann_df)
    for idx, row in t_ann_df_c.iterrows():
        assert idx + 1 == row['id']
    return _aug_img_df, _ann_df_c

# create annotatuion file

### extract data from these df:

1. clt_cp
2. ecp_quantile_95
3. ch_quantile_95
4. wp_quantile_95

### select %15 from each validation and train set for testing

In [53]:
cp_file = '/home/test_pc/mmlab/mmdetection/datasets/cityperson_train.json'
f = open(cp_file)
cp_data = json.load(f)
# cp_licenses = cp_data['licenses']
cp_categories = cp_data['categories']
cp_images = cp_data['images']
cp_annotations = cp_data['annotations']
cp_img_df = pd.DataFrame(cp_images)
cp_ann_df = pd.DataFrame(cp_annotations)

In [54]:
set1 = set(cp_img_df.file_name.to_list())

In [55]:
aug_img_df = copy.deepcopy(cp_img_df)
ann_df_c = copy.deepcopy(cp_ann_df)
print(aug_img_df.shape[0])
print(ann_df_c.shape[0])

2529
23429


In [56]:
cp_info = copy.deepcopy(cp_data['info'])
cp_cat = copy.deepcopy(cp_data['categories'])
cp_data['categories']

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]

### test anno

In [57]:
test_file = '/home/test_pc/mmlab/mmdetection/datasets/cityperson_test.json'
f = open(test_file)
t_data = json.load(f)
# cp_licenses = cp_data['licenses']
t_categories = t_data['categories']
t_images = t_data['images']
t_annotations = t_data['annotations']
t_img_df = pd.DataFrame(t_images)
t_ann_df = pd.DataFrame(t_annotations)

In [58]:
set2 = set(t_img_df.file_name.to_list())
print("set1 intersection set2 : ", set1.intersection(set2))

set1 intersection set2 :  set()


In [59]:
t_aug_img_df = copy.deepcopy(t_img_df)
t_ann_df_c = copy.deepcopy(t_ann_df)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

446
4341


In [60]:
t_aug_img_df.shape[0]

446

In [61]:
t_ann_df_c.shape[0]

4341

## ECP 

In [62]:
ecp_file = 'json_files/EuroCity/day_train_all.json'
f = open(ecp_file)
ecp_data = json.load(f)
ecp_licenses = ecp_data['licenses']
ecp_categories = ecp_data['categories']
ecp_images = ecp_data['images']
ecp_annotations = ecp_data['annotations']
ecp_img_df = pd.DataFrame(ecp_images)
ecp_ann_df = pd.DataFrame(ecp_annotations)
ecp_img_df.shape[0]

23892

In [63]:
print(ecp_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [64]:
for idx, row in ecp_img_df.iterrows():
    ecp_img_df.at[ecp_img_df.index[idx], 'file_name'] = 'EuroCity/' + ecp_img_df.at[ecp_img_df.index[idx], 'file_name']
ecp_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
1,2,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
2,3,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
3,4,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
4,5,EuroCity/ECP/day/img/train/wuerzburg/wuerzburg...,1920,1024,2019-11-03 07:44:18.143034,1,,
...,...,...,...,...,...,...,...,...
23887,23888,EuroCity/ECP/day/img/train/potsdam/potsdam_004...,1920,1024,2019-11-03 07:44:18.143034,1,,
23888,23889,EuroCity/ECP/day/img/train/potsdam/potsdam_000...,1920,1024,2019-11-03 07:44:18.143034,1,,
23889,23890,EuroCity/ECP/day/img/train/potsdam/potsdam_005...,1920,1024,2019-11-03 07:44:18.143034,1,,
23890,23891,EuroCity/ECP/day/img/train/potsdam/potsdam_003...,1920,1024,2019-11-03 07:44:18.143034,1,,


In [65]:
im_list = list(set([item for sublist in ecp_cp['images'].to_list() for item in sublist]))
len(im_list)

1194

In [66]:
train_img_ecp, val_img_ecp, test_img_ecp = splite_data(ecp_cp, train_size=0.73, val_size=0.14, test_size=0.13)
index_id_list = find_index_of_images(ecp_img_df, image_name_list=train_img_ecp)

selected_ecp_img_df = create_img_df(ecp_img_df, index_id_list)
selected_ecp_ann_df = create_ann_df(ecp_ann_df, index_id_list)
print(selected_ecp_ann_df.shape[0])
print(selected_ecp_img_df.shape[0])

7814
871


In [67]:
train_img_ecp

['milano_00157',
 'roma_00857',
 'bologna_01207',
 'bratislava_00534',
 'zagreb_00066',
 'barcelona_01256',
 'prague_00672',
 'berlin_00247',
 'prague_00129',
 'wuerzburg_00495',
 'lyon_00153',
 'pisa_00248',
 'amsterdam_00192',
 'zagreb_00680',
 'roma_00582',
 'amsterdam_00628',
 'ulm_00484',
 'ulm_00276',
 'ulm_00145',
 'stuttgart_00407',
 'dresden_00570',
 'ljubljana_00879',
 'montpellier_00366',
 'nuernberg_00132',
 'brno_00537',
 'berlin_00013',
 'bologna_01209',
 'ljubljana_00782',
 'ulm_00024',
 'ulm_00093',
 'zagreb_00600',
 'firenze_00459',
 'ulm_00402',
 'toulouse_00311',
 'roma_00593',
 'firenze_00779',
 'prague_00703',
 'budapest_00353',
 'prague_00021',
 'marseille_00881',
 'hamburg_00483',
 'amsterdam_00399',
 'roma_00359',
 'budapest_00794',
 'torino_00979',
 'zagreb_00297',
 'potsdam_00447',
 'szczecin_00188',
 'torino_00035',
 'berlin_00124',
 'pisa_00516',
 'toulouse_00579',
 'amsterdam_01022',
 'milano_00286',
 'marseille_00939',
 'prague_01120',
 'milano_00873',
 'b

In [68]:
pd.set_option('display.max_colwidth', None)

In [69]:
selected_ecp_img_df.file_name

10508            EuroCity/ECP/day/img/train/milano/milano_00157.png
9458                 EuroCity/ECP/day/img/train/roma/roma_00857.png
18382          EuroCity/ECP/day/img/train/bologna/bologna_01207.png
14770    EuroCity/ECP/day/img/train/bratislava/bratislava_00534.png
6209             EuroCity/ECP/day/img/train/zagreb/zagreb_00066.png
                                    ...                            
17754                  EuroCity/ECP/day/img/train/ulm/ulm_00614.png
23057      EuroCity/ECP/day/img/train/nuernberg/nuernberg_00653.png
14263          EuroCity/ECP/day/img/train/hamburg/hamburg_00068.png
10393            EuroCity/ECP/day/img/train/milano/milano_00452.png
10356            EuroCity/ECP/day/img/train/milano/milano_00210.png
Name: file_name, Length: 871, dtype: object

In [70]:
print(len(train_img_ecp))
len(val_img_ecp)

871


168

len(test_img_ecp)

In [71]:
print(f'add #{len(train_img_ecp)} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_ecp_ann_df = uodate_id_in_image_ann(selected_ecp_img_df, aug_img_df, selected_ecp_ann_df)
print(f'train img df after aug shape: {aug_img_df.shape[0]}')

add #871 images to #2529 train set
train img df after aug shape: 3400


In [72]:
selected_ecp_ann_df = add_area(selected_ecp_ann_df)
print(f'add #{selected_ecp_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')
ann_df_c = concat_ann_df(ann_df_c, selected_ecp_ann_df)
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')
ann_df_c

add #7814 annotations to #23429 train set
train annotations #31243 for train set after augmentation


,id,image_id,category_id,iscrowd,bbox,width,height,area
0,1,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540
2,3,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72
3,4,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760
4,5,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224
...,...,...,...,...,...,...,...,...
31238,31239,2899,1,True,"[1074.0, 526.0, 202.0, 124.0]",1920,1024,25048
31239,31240,2899,1,True,"[897.0, 553.0, 43.0, 33.0]",1920,1024,1419
31240,31241,2899,1,True,"[827.0, 538.0, 12.0, 53.0]",1920,1024,636
31241,31242,2899,1,True,"[1076.0, 537.0, 8.0, 23.0]",1920,1024,184


#### test anno

In [73]:
print(f'test img df before aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df before aug shape: {t_ann_df_c.shape[0]}')
t_aug_img_df, t_ann_df_c = _ugment_df(ecp_img_df, ecp_ann_df, test_img_ecp, t_aug_img_df, t_ann_df_c)
print(f'test img df after aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df after aug shape: {t_ann_df_c.shape[0]}')

test img df before aug shape: 446
test ann df before aug shape: 4341
you will add 155 to images
you will add 1398 to anno
test img df after aug shape: 601
test ann df after aug shape: 5739


### crowdHuman

In [74]:
ch_file = 'json_files/CrowdHuman/train.json'

f = open(ch_file)
ch_data = json.load(f)
# ch_licenses = ch_data['licenses']
ch_categories = ch_data['categories']
ch_images = ch_data['images']
ch_annotations = ch_data['annotations']
ch_img_df = pd.DataFrame(ch_images)
ch_ann_df = pd.DataFrame(ch_annotations)

In [75]:
print(ch_data['categories'])

[{'supercategory': 'none', 'id': 1, 'name': 'person'}, {'supercategory': 'none', 'id': 2, 'name': 'mask'}]


In [76]:
bb_df = copy.deepcopy(ch_ann_df[ch_ann_df.category_id == 1])
ch_ann_df = bb_df.copy()
ch_ann_df

,area,iscrowd,image_id,bbox,hbox,vbox,category_id,id,ignore
0,86523,0,1,"[61, 123, 191, 453]","[123, 129, 63, 64]","[62, 126, 154, 446]",1,1,0
1,92378,0,1,"[165, 95, 187, 494]","[214, 97, 58, 74]","[175, 95, 140, 487]",1,2,0
2,96135,0,1,"[236, 104, 195, 493]","[318, 109, 58, 68]","[260, 106, 170, 487]",1,3,0
3,85852,0,1,"[452, 110, 169, 508]","[486, 119, 61, 74]","[455, 113, 141, 501]",1,4,0
4,62103,0,1,"[520, 95, 163, 381]","[559, 105, 53, 57]","[553, 98, 70, 118]",1,5,0
...,...,...,...,...,...,...,...,...,...
438781,286612,0,15000,"[3393, 590, 316, 907]","[3518, 595, 114, 133]","[3393, 590, 316, 907]",1,438782,0
438782,81890,0,15000,"[3448, 607, 190, 431]","[3511, 610, 93, 104]","[3452, 608, 184, 430]",1,438783,0
438783,240570,0,15000,"[3684, 554, 297, 810]","[3824, 559, 96, 105]","[3795, 559, 156, 794]",1,438784,0
438784,234855,0,15000,"[3797, 536, 255, 921]","[3909, 539, 89, 113]","[3889, 535, 112, 169]",1,438785,0


In [77]:
for idx, row in ch_img_df.iterrows():
    ch_img_df.at[ch_img_df.index[idx], 'file_name'] = 'CrowdHuman/Images/' + ch_img_df.at[ch_img_df.index[idx], 'file_name']
ch_img_df

,file_name,height,width,id
0,"CrowdHuman/Images/284193,faa9000f2678b5e.jpg",683,1024,1
1,"CrowdHuman/Images/273275,cd061000af95f691.jpg",480,600,2
2,"CrowdHuman/Images/273278,8d231000e09fc133.jpg",1017,1300,3
3,"CrowdHuman/Images/283554,2cd4d0007833968d.jpg",954,1300,4
4,"CrowdHuman/Images/283554,37ba1000a6ca8c4d.jpg",533,800,5
...,...,...,...,...
14995,"CrowdHuman/Images/273275,874d9000417e16ed.jpg",1192,1800,14996
14996,"CrowdHuman/Images/273278,d329e000260a8cc2.jpg",600,1800,14997
14997,"CrowdHuman/Images/282555,c5fd20007faf5f84.jpg",575,1024,14998
14998,"CrowdHuman/Images/273275,10b78d0006d7d7b9c.jpg",462,1838,14999


In [78]:
ch_selected_df = copy.deepcopy(ch_quantile_95)

In [79]:
ch_img_df

,file_name,height,width,id
0,"CrowdHuman/Images/284193,faa9000f2678b5e.jpg",683,1024,1
1,"CrowdHuman/Images/273275,cd061000af95f691.jpg",480,600,2
2,"CrowdHuman/Images/273278,8d231000e09fc133.jpg",1017,1300,3
3,"CrowdHuman/Images/283554,2cd4d0007833968d.jpg",954,1300,4
4,"CrowdHuman/Images/283554,37ba1000a6ca8c4d.jpg",533,800,5
...,...,...,...,...
14995,"CrowdHuman/Images/273275,874d9000417e16ed.jpg",1192,1800,14996
14996,"CrowdHuman/Images/273278,d329e000260a8cc2.jpg",600,1800,14997
14997,"CrowdHuman/Images/282555,c5fd20007faf5f84.jpg",575,1024,14998
14998,"CrowdHuman/Images/273275,10b78d0006d7d7b9c.jpg",462,1838,14999


In [80]:
train_img_ch, val_img_ch, test_img_ch = splite_data(ch_selected_df, train_size=0.7, val_size=0.15, test_size=0.15)
index_id_list = find_index_of_images(ch_img_df, image_name_list=train_img_ch)
selected_ch_img_df = create_img_df(ch_img_df, index_id_list)
selected_ch_ann_df = create_ann_df(ch_ann_df, index_id_list)
print(selected_ch_img_df.shape[0])
selected_ch_ann_df.shape[0]

436


7077

In [81]:
len(test_img_ch)

94

In [82]:
print(len(train_img_ch))
len(val_img_ch)

436


94

In [83]:
print(f'add #{len(train_img_ch)} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_ch_ann_df = uodate_id_in_image_ann(selected_ch_img_df, aug_img_df, selected_ch_ann_df, dt='ch')
print(f'train img df after aug shape: {aug_img_df.shape[0]}')

add #436 images to #3400 train set
train img df after aug shape: 3836


In [84]:
selected_ch_ann_df.shape[0]

7077

In [85]:
selected_ch_ann_df = add_area(selected_ch_ann_df)
print(f'add #{selected_ch_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')
ann_df_c = concat_ann_df(ann_df_c, selected_ch_ann_df)
ann_df_c = ann_df_c.drop(['hbox', 'vbox'], axis = 1)
print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')

add #7077 annotations to #31243 train set
train annotations #38320 for train set after augmentation


#### test anno

In [86]:
print(f'test img df before aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df before aug shape: {t_ann_df_c.shape[0]}')
t_aug_img_df, t_ann_df_c = _ugment_df(ch_img_df, ch_ann_df, test_img_ch, t_aug_img_df, t_ann_df_c)
t_ann_df_c = t_ann_df_c.drop(['hbox', 'vbox'], axis = 1)
print(f'test img df after aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df after aug shape: {t_ann_df_c.shape[0]}')

test img df before aug shape: 601
test ann df before aug shape: 5739
you will add 94 to images
you will add 1357 to anno
test img df after aug shape: 695
test ann df after aug shape: 7096


### wheelchair

In [87]:
wheel_file = '/home/test_pc/mmlab/mmdetection/datasets/wheelchair/train.json'

f = open(wheel_file)
wheel_data = json.load(f)
wheel_categories = wheel_data['categories']
wheel_images = wheel_data['images']
wheel_annotations = wheel_data['annotations']
wheel_img_df = pd.DataFrame(wheel_images)
wheel_ann_df = pd.DataFrame(wheel_annotations)
print(wheel_img_df.shape[0])

20


In [88]:
wheel_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,wheelchair/images/train/14b2ccfde47a3703.jpg,1024,683,2021-04-18 13:48:33.015842,1,,
1,2,wheelchair/images/train/2fc7344cc84236bb.jpg,1024,678,2021-04-18 13:48:33.015842,1,,
2,3,wheelchair/images/train/11e8b4148d34f05b.jpg,1024,768,2021-04-18 13:48:33.015842,1,,
3,4,wheelchair/images/train/7beb0ed8e41a4a40.jpg,681,1024,2021-04-18 13:48:33.015842,1,,
4,5,wheelchair/images/train/0f137232e9e731e2.jpg,1024,683,2021-04-18 13:48:33.015842,1,,
5,6,wheelchair/images/train/6a389788ad2e0425.jpg,1024,813,2021-04-18 13:48:33.015842,1,,
6,7,wheelchair/images/train/33fb23e4317038d8.jpg,683,1024,2021-04-18 13:48:33.015842,1,,
7,8,wheelchair/images/train/2b29b0237fb48e9e.jpg,684,1024,2021-04-18 13:48:33.015842,1,,
8,9,wheelchair/images/train/5c96e11bc62d3cfa.jpg,1024,683,2021-04-18 13:48:33.015842,1,,
9,10,wheelchair/images/train/75b58881d868d7db.jpg,747,1024,2021-04-18 13:48:33.015842,1,,


In [89]:
print(wheel_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [90]:
wheel_ann_df = add_area(wheel_ann_df)

print(wheel_ann_df.shape[0])

20


In [91]:
selected_img_df = copy.deepcopy(wheel_img_df)
selected_ann_df = copy.deepcopy(wheel_ann_df)

In [92]:
print(f'test img df before aug shape: {aug_img_df.shape[0]}')
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)
print(f'test img df after aug shape: {aug_img_df.shape[0]}')

test img df before aug shape: 3836
test img df after aug shape: 3856


In [93]:
print(ann_df_c.shape[0])
print(selected_ann_df.shape[0])
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
# ann_df_c = copy.deepcopy(wheel_ann_df)
ann_df_c
print(ann_df_c.shape[0])

38320
20
38340


In [94]:
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

In [95]:
! ls /home/test_pc/mmlab/mmdetection/datasets/rand_cal/val.json

/home/test_pc/mmlab/mmdetection/datasets/rand_cal/val.json


#### test anno

In [96]:
wheel_file = '/home/test_pc/mmlab/mmdetection/datasets/wheelchair/test.json'

f = open(wheel_file)
wt_data = json.load(f)
wt_categories = wheel_data['categories']
wt_images = wt_data['images']
wt_annotations = wt_data['annotations']
wt_img_df = pd.DataFrame(wt_images)
wt_ann_df = pd.DataFrame(wt_annotations)
print(wt_img_df.shape[0])
print(wt_ann_df.shape[0])

10
11


In [97]:
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])
s_img_df = copy.deepcopy(wt_img_df)
s_ann_df = copy.deepcopy(wt_ann_df)
t_aug_img_df, s_ann_df = uodate_id_in_image_ann(s_img_df, t_aug_img_df, s_ann_df)
t_ann_df_c = concat_ann_df(t_ann_df_c, s_ann_df)
print(t_aug_img_df.shape[0])
print(t_ann_df_c.shape[0])

695
7096
705
7107


### WP

In [98]:
wp1_file = 'json_files/WiderPerson/train.json'

f = open(wp1_file)
wp_data = json.load(f)
wp_categories = wp_data['categories']
wp_images = wp_data['images']
wp_annotations = wp_data['annotations']
wp_img_df = pd.DataFrame(wp_images)
wp_ann_df = pd.DataFrame(wp_annotations)
wp_img_df.shape[0]

8000

In [99]:
wp_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,WiderPerson/Images/000040.jpg,550,413,2021-03-16 23:06:50.130304,1,,
1,2,WiderPerson/Images/000042.jpg,551,350,2021-03-16 23:06:50.130304,1,,
2,3,WiderPerson/Images/000051.jpg,450,800,2021-03-16 23:06:50.130304,1,,
3,4,WiderPerson/Images/000081.jpg,600,395,2021-03-16 23:06:50.130304,1,,
4,5,WiderPerson/Images/000089.jpg,600,400,2021-03-16 23:06:50.130304,1,,
...,...,...,...,...,...,...,...,...
7995,7996,WiderPerson/Images/020544.jpg,397,255,2021-03-16 23:06:50.130304,1,,
7996,7997,WiderPerson/Images/020545.jpg,700,476,2021-03-16 23:06:50.130304,1,,
7997,7998,WiderPerson/Images/020547.jpg,1272,784,2021-03-16 23:06:50.130304,1,,
7998,7999,WiderPerson/Images/020548.jpg,4208,2368,2021-03-16 23:06:50.130304,1,,


In [100]:
print(wp_data['categories'])

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]


In [101]:
train_img_wp, val_img_wp, test_img_wp = splite_data(wp_quantile_95, train_size=0.7, val_size=0.15, test_size=0.15)
index_id_list = find_index_of_images(wp_img_df, image_name_list=train_img_wp)

In [102]:
len(set(train_img_wp))

351

In [103]:
len(set(val_img_wp))

76

In [104]:
len(set(test_img_wp))

75

In [105]:
selected_wp_img_df = create_img_df(wp_img_df, index_id_list)
selected_wp_img_df.shape[0]

351

In [106]:
selected_wp_ann_df = create_ann_df(wp_ann_df, index_id_list)
selected_wp_ann_df = add_area(selected_wp_ann_df)
selected_wp_ann_df.shape[0]

9586

In [107]:
print(f'add #{len(train_img_wp)} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_wp_ann_df = uodate_id_in_image_ann(selected_wp_img_df, aug_img_df, selected_wp_ann_df)
print(f'train img df after aug shape: {aug_img_df.shape[0]}')

add #351 images to #3856 train set
train img df after aug shape: 4207


In [108]:
selected_wp_ann_df.shape[0]

9586

In [109]:
print(f'add #{selected_wp_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')
ann_df_c = concat_ann_df(ann_df_c, selected_wp_ann_df)
# ann_df_c

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']
print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')

add #9586 annotations to #38340 train set
train annotations #47926 for train set after augmentation


In [110]:
ann_df_c.shape[0]

47926

#### test anno

In [111]:
print(f'test img df before aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df before aug shape: {t_ann_df_c.shape[0]}')
t_aug_img_df, t_ann_df_c = _ugment_df(wp_img_df, wp_ann_df, test_img_wp, t_aug_img_df, t_ann_df_c)
print(f'test img df after aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df after aug shape: {t_ann_df_c.shape[0]}')

test img df before aug shape: 705
test ann df before aug shape: 7107
you will add 75 to images
you will add 2396 to anno
test img df after aug shape: 780
test ann df after aug shape: 9503


#### selected caltech

In [112]:
ca_file = '/home/test_pc/mmlab/mmdetection/datasets/selected_caltech/train.json'

f = open(ca_file)
cl_data = json.load(f)
cl_categories = cl_data['categories']
cl_images = cl_data['images']
cl_annotations = cl_data['annotations']
cl_img_df = pd.DataFrame(cl_images)
cl_ann_df = pd.DataFrame(cl_annotations)
cl_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,selected_caltech/images/train/set00_V014_1538.png,640,480,2021-04-17 19:49:08.481689,1,,
1,2,selected_caltech/images/train/set00_V013_1429.png,640,480,2021-04-17 19:49:08.481689,1,,
2,3,selected_caltech/images/train/set00_V010_231.png,640,480,2021-04-17 19:49:08.481689,1,,
3,4,selected_caltech/images/train/set04_V011_64.png,640,480,2021-04-17 19:49:08.481689,1,,
4,5,selected_caltech/images/train/set03_V010_1055.png,640,480,2021-04-17 19:49:08.481689,1,,
...,...,...,...,...,...,...,...,...
196,197,selected_caltech/images/train/set05_V003_41.png,640,480,2021-04-17 19:49:08.481689,1,,
197,198,selected_caltech/images/train/set05_V011_1510.png,640,480,2021-04-17 19:49:08.481689,1,,
198,199,selected_caltech/images/train/set00_V000_1110.png,640,480,2021-04-17 19:49:08.481689,1,,
199,200,selected_caltech/images/train/set02_V009_56.png,640,480,2021-04-17 19:49:08.481689,1,,


In [113]:
cl_ann_df = add_area(cl_ann_df)

In [114]:
selected_img_df = copy.deepcopy(cl_img_df)
selected_ann_df = copy.deepcopy(cl_ann_df)
aug_img_df.shape[0] + selected_img_df.shape[0]

4408

In [115]:
print(f'add #{selected_img_df.shape[0]} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)
print(f'train img df after aug shape: {aug_img_df.shape[0]}')

add #201 images to #4207 train set
train img df after aug shape: 4408


In [116]:
aug_img_df.shape[0]

4408

In [117]:
selected_ann_df.shape[0] + ann_df_c.shape[0]

48235

In [118]:
print(f'add #{selected_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

ann_df_c

add #309 annotations to #47926 train set


,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,1,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325,NaN
1,2,1,1,1,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540,NaN
2,3,1,1,1,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72,NaN
3,4,2,1,0,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760,NaN
4,5,2,1,0,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224,NaN
...,...,...,...,...,...,...,...,...,...
48230,48231,4401,1,0,"[149, 150, 24, 50]",640,480,1200,NaN
48231,48232,4402,1,0,"[304, 168, 22, 42]",640,480,924,NaN
48232,48233,4403,1,0,"[41, 173, 45, 72]",640,480,3240,NaN
48233,48234,4406,1,0,"[605, 163, 18, 41]",640,480,738,NaN


In [119]:
ann_df_c["iscrowd"].replace({1: True, 0:False}, inplace=True)
ann_df_c['ignore'] = ann_df_c['ignore'].fillna(0)
# ann_df_c = ann_df_c.drop(['hbox', 'vbox'], axis = 1)
ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325,0.0
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540,0.0
2,3,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72,0.0
3,4,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760,0.0
4,5,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224,0.0
...,...,...,...,...,...,...,...,...,...
48230,48231,4401,1,False,"[149, 150, 24, 50]",640,480,1200,0.0
48231,48232,4402,1,False,"[304, 168, 22, 42]",640,480,924,0.0
48232,48233,4403,1,False,"[41, 173, 45, 72]",640,480,3240,0.0
48233,48234,4406,1,False,"[605, 163, 18, 41]",640,480,738,0.0


In [120]:
ann_df_c[ann_df_c.category_id != 1]

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore


In [121]:
wheel_data['categories']

[{'id': 1, 'name': 'pedestrain', 'supercategory': 'pedestrain'}]

#### test anno

In [122]:
ct_file = '/home/test_pc/mmlab/mmdetection/datasets/selected_caltech/test.json'

f = open(ct_file)
ct_data = json.load(f)
ct_categories = ct_data['categories']
ct_images = ct_data['images']
ct_annotations = ct_data['annotations']

ct_img_df = pd.DataFrame(ct_images)
ct_ann_df = pd.DataFrame(ct_annotations)

print(ct_img_df.shape[0])
print(ct_ann_df.shape[0])

print(f'test img df before aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df before aug shape: {t_ann_df_c.shape[0]}')
s_img_df = copy.deepcopy(ct_img_df)
s_ann_df = copy.deepcopy(ct_ann_df)

t_aug_img_df, s_ann_df = uodate_id_in_image_ann(s_img_df, t_aug_img_df, s_ann_df)
t_ann_df_c = concat_ann_df(t_ann_df_c, s_ann_df)
print(f'test img df after aug shape: {t_aug_img_df.shape[0]}')
print(f'test ann df after aug shape: {t_ann_df_c.shape[0]}')

43
83
test img df before aug shape: 780
test ann df before aug shape: 9503
test img df after aug shape: 823
test ann df after aug shape: 9586


In [123]:
t_ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,1,"[942.0, 406.0, 18.0, 44.0]",2048.0,1024.0,792,NaN
1,2,1,1,1,"[925.0, 404.0, 19.0, 46.0]",2048.0,1024.0,874,NaN
2,3,1,1,1,"[856.0, 415.0, 35.0, 83.0]",2048.0,1024.0,2905,NaN
3,4,3,1,0,"[325.0, 417.0, 51.0, 123.0]",2048.0,1024.0,6273,NaN
4,5,3,1,1,"[1017.0, 408.0, 15.0, 38.0]",2048.0,1024.0,570,NaN
...,...,...,...,...,...,...,...,...,...
9581,9582,819,1,0,"[550, 193, 24, 50]",640.0,480.0,NaN,NaN
9582,9583,820,1,0,"[14, 187, 26, 59]",640.0,480.0,NaN,NaN
9583,9584,821,1,0,"[592, 160, 18, 39]",640.0,480.0,NaN,NaN
9584,9585,821,1,0,"[575, 151, 14, 53]",640.0,480.0,NaN,NaN


In [124]:
t_aug_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000113_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000122_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000114_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000064_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000077_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
...,...,...,...,...,...,...,...,...
818,819,selected_caltech/images/test/set05_V005_469.png,640,480,2021-04-17 19:49:18.073247,1.0,,
819,820,selected_caltech/images/test/set02_V003_147.png,640,480,2021-04-17 19:49:18.073247,1.0,,
820,821,selected_caltech/images/test/set03_V009_1727.png,640,480,2021-04-17 19:49:18.073247,1.0,,
821,822,selected_caltech/images/test/set02_V009_176.png,640,480,2021-04-17 19:49:18.073247,1.0,,


### Save to Json

In [125]:
t_ann_df_c["iscrowd"].replace({1: True, 0:False}, inplace=True)
# t_ann_df_c['ignore'] = t_ann_df_c['ignore'].fillna(0)
t_ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,True,"[942.0, 406.0, 18.0, 44.0]",2048.0,1024.0,792,NaN
1,2,1,1,True,"[925.0, 404.0, 19.0, 46.0]",2048.0,1024.0,874,NaN
2,3,1,1,True,"[856.0, 415.0, 35.0, 83.0]",2048.0,1024.0,2905,NaN
3,4,3,1,False,"[325.0, 417.0, 51.0, 123.0]",2048.0,1024.0,6273,NaN
4,5,3,1,True,"[1017.0, 408.0, 15.0, 38.0]",2048.0,1024.0,570,NaN
...,...,...,...,...,...,...,...,...,...
9581,9582,819,1,False,"[550, 193, 24, 50]",640.0,480.0,NaN,NaN
9582,9583,820,1,False,"[14, 187, 26, 59]",640.0,480.0,NaN,NaN
9583,9584,821,1,False,"[592, 160, 18, 39]",640.0,480.0,NaN,NaN
9584,9585,821,1,False,"[575, 151, 14, 53]",640.0,480.0,NaN,NaN


In [126]:
ann_df_c["iscrowd"].replace({1: True, 0:False}, inplace=True)
# ann_df_c['ignore'] = ann_df_c['ignore'].fillna(0)
ann_df_c

,id,image_id,category_id,iscrowd,bbox,width,height,area,ignore
0,1,1,1,True,"[1034.0, 366.0, 31.0, 75.0]",2048,1024,2325,0.0
1,2,1,1,True,"[1911.0, 359.0, 10.0, 54.0]",2048,1024,540,0.0
2,3,1,1,True,"[1137.0, 307.0, 8.0, 9.0]",2048,1024,72,0.0
3,4,2,1,False,"[602.0, 413.0, 60.0, 146.0]",2048,1024,8760,0.0
4,5,2,1,False,"[281.0, 381.0, 89.0, 216.0]",2048,1024,19224,0.0
...,...,...,...,...,...,...,...,...,...
48230,48231,4401,1,False,"[149, 150, 24, 50]",640,480,1200,0.0
48231,48232,4402,1,False,"[304, 168, 22, 42]",640,480,924,0.0
48232,48233,4403,1,False,"[41, 173, 45, 72]",640,480,3240,0.0
48233,48234,4406,1,False,"[605, 163, 18, 41]",640,480,738,0.0


In [127]:
aug_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000117_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000080_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000128_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000103_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000063_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
...,...,...,...,...,...,...,...,...
4403,4404,selected_caltech/images/train/set05_V003_41.png,640,480,2021-04-17 19:49:08.481689,1.0,,
4404,4405,selected_caltech/images/train/set05_V011_1510.png,640,480,2021-04-17 19:49:08.481689,1.0,,
4405,4406,selected_caltech/images/train/set00_V000_1110.png,640,480,2021-04-17 19:49:08.481689,1.0,,
4406,4407,selected_caltech/images/train/set02_V009_56.png,640,480,2021-04-17 19:49:08.481689,1.0,,


In [128]:
print(f"CityPersons: #{aug_img_df[aug_img_df['file_name'].str.contains('CityPersons')].shape[0]}")
# selected_caltech
print(f"selected_caltech: #{aug_img_df[aug_img_df['file_name'].str.contains('selected_caltech')].shape[0]}")
# WiderPerson
print(f"WiderPerson: #{aug_img_df[aug_img_df['file_name'].str.contains('WiderPerson')].shape[0]}")
# wheelchair
print(f"wheelchair: #{aug_img_df[aug_img_df['file_name'].str.contains('wheelchair')].shape[0]}")
# CrowdHuman
print(f"CrowdHuman: #{aug_img_df[aug_img_df['file_name'].str.contains('CrowdHuman')].shape[0]}")
# EuroCity
print(f"EuroCity: #{aug_img_df[aug_img_df['file_name'].str.contains('EuroCity')].shape[0]}")

CityPersons: #2529
selected_caltech: #201
WiderPerson: #351
wheelchair: #20
CrowdHuman: #436
EuroCity: #871


In [129]:
t_aug_img_df

,id,file_name,width,height,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000113_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
1,2,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000122_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
2,3,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000114_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
3,4,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000064_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
4,5,CityPersons/leftImg8bit_trainvaltest//leftImg8bit/train/weimar/weimar_000077_000019_leftImg8bit.png,2048,1024,2019-07-25 11:20:43.195846,1.0,,
...,...,...,...,...,...,...,...,...
818,819,selected_caltech/images/test/set05_V005_469.png,640,480,2021-04-17 19:49:18.073247,1.0,,
819,820,selected_caltech/images/test/set02_V003_147.png,640,480,2021-04-17 19:49:18.073247,1.0,,
820,821,selected_caltech/images/test/set03_V009_1727.png,640,480,2021-04-17 19:49:18.073247,1.0,,
821,822,selected_caltech/images/test/set02_V009_176.png,640,480,2021-04-17 19:49:18.073247,1.0,,


In [130]:
print(f"CityPersons: #{t_aug_img_df[t_aug_img_df['file_name'].str.contains('CityPersons')].shape[0]}")
# selected_caltech
print(f"selected_caltech: #{t_aug_img_df[t_aug_img_df['file_name'].str.contains('selected_caltech')].shape[0]}")
# WiderPerson
print(f"WiderPerson: #{t_aug_img_df[t_aug_img_df['file_name'].str.contains('WiderPerson')].shape[0]}")
# wheelchair
print(f"wheelchair: #{t_aug_img_df[t_aug_img_df['file_name'].str.contains('wheelchair')].shape[0]}")
# CrowdHuman
print(f"CrowdHuman: #{t_aug_img_df[t_aug_img_df['file_name'].str.contains('CrowdHuman')].shape[0]}")
# EuroCity
print(f"EuroCity: #{t_aug_img_df[t_aug_img_df['file_name'].str.contains('EuroCity')].shape[0]}")

CityPersons: #446
selected_caltech: #43
WiderPerson: #75
wheelchair: #10
CrowdHuman: #94
EuroCity: #155


In [131]:
aug_data_test = dict()
aug_data_test['info'] = cp_data['info']
# aug_data_test['licenses'] = cp_data['licenses']
aug_data_test['categories'] = cp_data['categories']

aug_data_test['images'] = t_aug_img_df.to_dict('records')
aug_data_test['annotations'] = t_ann_df_c.to_dict('records')

with open('/home/test_pc/mmlab/mmdetection/datasets/aug_test.json', 'w') as fp:
    json.dump(aug_data_test, fp)

In [132]:
aug_test = '/home/test_pc/mmlab/mmdetection/datasets/aug_test.json'
f = open(aug_test)
aug_test_data = json.load(f)
# aug_test_licenses = aug_test_data['licenses']
aug_test_cat = aug_test_data['categories']
aug_test_images = aug_test_data['images']
aug_test_annotations = aug_test_data['annotations']
aug_test_img_df = pd.DataFrame(aug_test_images)
aug_test_ann_df = pd.DataFrame(aug_test_annotations)

print(f"CityPersons: #{aug_test_img_df[aug_test_img_df['file_name'].str.contains('CityPersons')].shape[0]}")
# selected_caltech
print(f"selected_caltech: #{aug_test_img_df[aug_test_img_df['file_name'].str.contains('selected_caltech')].shape[0]}")
# WiderPerson
print(f"WiderPerson: #{aug_test_img_df[aug_test_img_df['file_name'].str.contains('WiderPerson')].shape[0]}")
# wheelchair
print(f"wheelchair: #{aug_test_img_df[aug_test_img_df['file_name'].str.contains('wheelchair')].shape[0]}")
# CrowdHuman
print(f"CrowdHuman: #{aug_test_img_df[aug_test_img_df['file_name'].str.contains('CrowdHuman')].shape[0]}")
# EuroCity
print(f"EuroCity: #{aug_test_img_df[aug_test_img_df['file_name'].str.contains('EuroCity')].shape[0]}")

CityPersons: #446
selected_caltech: #43
WiderPerson: #75
wheelchair: #10
CrowdHuman: #94
EuroCity: #155


In [133]:
aug_data_train = dict()
aug_data_train['info'] = cp_data['info']
# aug_data_train['licenses'] = cp_data['licenses']
aug_data_train['categories'] = cp_data['categories']

aug_data_train['images'] = aug_img_df.to_dict('records')
aug_data_train['annotations'] = ann_df_c.to_dict('records')

In [134]:
with open('/home/test_pc/mmlab/mmdetection/datasets/cp_train.json', 'w') as fp:
    json.dump(aug_data_train, fp)

In [135]:
cp_train = '/home/test_pc/mmlab/mmdetection/datasets/cp_train.json'
f = open(cp_train)
cp_train_data = json.load(f)
# cp_train_licenses = cp_train_data['licenses']
cp_train_cat = cp_train_data['categories']
cp_train_images = cp_train_data['images']
cp_train_annotations = cp_train_data['annotations']
cp_train_img_df = pd.DataFrame(cp_train_images)
cp_train_ann_df = pd.DataFrame(cp_train_annotations)

print(f"CityPersons: #{cp_train_img_df[cp_train_img_df['file_name'].str.contains('CityPersons')].shape[0]}")
# selected_caltech
print(f"selected_caltech: #{cp_train_img_df[cp_train_img_df['file_name'].str.contains('selected_caltech')].shape[0]}")
# WiderPerson
print(f"WiderPerson: #{cp_train_img_df[cp_train_img_df['file_name'].str.contains('WiderPerson')].shape[0]}")
# wheelchair
print(f"wheelchair: #{cp_train_img_df[cp_train_img_df['file_name'].str.contains('wheelchair')].shape[0]}")
# CrowdHuman
print(f"CrowdHuman: #{cp_train_img_df[cp_train_img_df['file_name'].str.contains('CrowdHuman')].shape[0]}")
# EuroCity
print(f"EuroCity: #{cp_train_img_df[cp_train_img_df['file_name'].str.contains('EuroCity')].shape[0]}")

CityPersons: #2529
selected_caltech: #201
WiderPerson: #351
wheelchair: #20
CrowdHuman: #436
EuroCity: #871


## Validation

In [136]:
cp_file = '/home/test_pc/mmlab/mmdetection/datasets/cityperson_val.json'
f = open(cp_file)
cp_data = json.load(f)
# cp_licenses = cp_data['licenses']
cp_categories = cp_data['categories']
cp_images = cp_data['images']
cp_annotations = cp_data['annotations']
cp_img_df = pd.DataFrame(cp_images)
cp_ann_df = pd.DataFrame(cp_annotations)
cp_ann_df

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1,"[1839, 382, 31, 71]",1024,2048,2201
1,2,1,1,0,0,"[947, 406, 17, 40]",1024,2048,680
2,3,1,1,0,1,"[990, 374, 14, 17]",1024,2048,238
3,4,1,1,0,1,"[1914, 73, 88, 90]",1024,2048,7920
4,5,1,1,0,1,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
4726,4727,423,1,0,1,"[1095, 419, 14, 22]",1024,2048,308
4727,4728,424,1,0,1,"[877, 435, 28, 17]",1024,2048,476
4728,4729,424,1,0,1,"[1370, 391, 26, 18]",1024,2048,468
4729,4730,424,1,0,1,"[1468, 338, 11, 13]",1024,2048,143


In [137]:
cp_img_df

,id,file_name,height,width
0,1,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_000294_leftImg8bit.png,1024,2048
1,2,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_000576_leftImg8bit.png,1024,2048
2,3,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_001016_leftImg8bit.png,1024,2048
3,4,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_001236_leftImg8bit.png,1024,2048
4,5,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_001751_leftImg8bit.png,1024,2048
...,...,...,...,...
420,421,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/munster_000168_000019_leftImg8bit.png,1024,2048
421,422,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/munster_000169_000019_leftImg8bit.png,1024,2048
422,423,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/munster_000170_000019_leftImg8bit.png,1024,2048
423,424,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/munster_000172_000019_leftImg8bit.png,1024,2048


In [138]:
aug_img_df = copy.deepcopy(cp_img_df)
ann_df_c = copy.deepcopy(cp_ann_df)

### ECP

In [139]:
index_id_list = find_index_of_images(ecp_img_df, image_name_list=val_img_ecp)

selected_img_df = create_img_df(ecp_img_df, index_id_list)

selected_ann_df = create_ann_df(ecp_ann_df, index_id_list)
selected_ann_df = add_area(selected_ann_df)
selected_ann_df

,id,image_id,category_id,iscrowd,bbox,width,height,area
89065,89066,11023,1,False,"[0.0, 508.0, 89.0, 283.0]",1920,1024,25187
89066,89067,11023,1,False,"[499.0, 547.0, 25.0, 70.0]",1920,1024,1750
89067,89068,11023,1,False,"[711.0, 551.0, 26.0, 84.0]",1920,1024,2184
89068,89069,11023,1,True,"[291.0, 583.0, 35.0, 67.0]",1920,1024,2345
89069,89070,11023,1,True,"[1247.0, 556.0, 74.0, 56.0]",1920,1024,4144
...,...,...,...,...,...,...,...,...
161133,161134,20048,1,True,"[963.0, 422.0, 9.0, 24.0]",1920,1024,216
161134,161135,20048,1,True,"[1108.0, 403.0, 126.0, 108.0]",1920,1024,13608
161135,161136,20048,1,True,"[1115.0, 404.0, 175.0, 120.0]",1920,1024,21000
161136,161137,20048,1,True,"[1003.0, 419.0, 13.0, 29.0]",1920,1024,377


In [140]:
selected_img_df.shape[0]

168

In [141]:
print(f'add #{selected_img_df.shape[0]} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)
print(f'train img df after aug shape: {aug_img_df.shape[0]}')

add #168 images to #425 train set
train img df after aug shape: 593


In [142]:
print(f'add #{selected_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')

ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
print(ann_df_c.shape[0])
for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')

add #1325 annotations to #4731 train set
6056
train annotations #6056 for train set after augmentation


### CH

In [143]:
index_id_list = find_index_of_images(ch_img_df, image_name_list=val_img_ch)

selected_img_df = create_img_df(ch_img_df, index_id_list)
selected_ann_df = create_ann_df(ch_ann_df, index_id_list)


selected_ann_df = add_area(selected_ann_df)

print(f'add #{selected_img_df.shape[0]} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df, dt='ch')
print(f'train img df after aug shape: {aug_img_df.shape[0]}')

print(f'add #{selected_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)
print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']


add #94 images to #593 train set
train img df after aug shape: 687
add #1657 annotations to #6056 train set
train annotations #7713 for train set after augmentation


In [144]:
ann_df_c = ann_df_c.drop(['hbox', 'vbox'], axis = 1)
ann_df_c

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1.0,"[1839, 382, 31, 71]",1024,2048,2201
1,2,1,1,0,0.0,"[947, 406, 17, 40]",1024,2048,680
2,3,1,1,0,1.0,"[990, 374, 14, 17]",1024,2048,238
3,4,1,1,0,1.0,"[1914, 73, 88, 90]",1024,2048,7920
4,5,1,1,0,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
7708,7709,618,1,0,0.0,"[818, 155, 91, 217]",641,960,19747
7709,7710,618,1,0,0.0,"[615, 118, 90, 239]",641,960,21510
7710,7711,618,1,0,0.0,"[427, 178, 66, 161]",641,960,10626
7711,7712,618,1,0,0.0,"[746, 116, 84, 251]",641,960,21084


### wheelchair

In [145]:
wheel_file = '/home/test_pc/mmlab/mmdetection/datasets/wheelchair/val.json'

f = open(wheel_file)
wheel_data = json.load(f)
wheel_categories = wheel_data['categories']
wheel_images = wheel_data['images']
wheel_annotations = wheel_data['annotations']
wheel_img_df = pd.DataFrame(wheel_images)
wheel_ann_df = pd.DataFrame(wheel_annotations)
wheel_img_df.shape[0]

11

In [146]:
wheel_ann_df = add_area(wheel_ann_df)
selected_img_df = copy.deepcopy(wheel_img_df)
selected_ann_df = copy.deepcopy(wheel_ann_df)
print(f'add #{selected_img_df.shape[0]} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)

print(f'train img df after aug shape: {aug_img_df.shape[0]}')


add #    id                                   file_name  width  height  \
0    1  wheelchair/images/val/d042a7f8c38def0a.jpg    768    1024   
1    2  wheelchair/images/val/c67beee99ba72705.jpg   1024     685   
2    3  wheelchair/images/val/b4df8bd4569e2bb5.jpg    683    1024   
3    4  wheelchair/images/val/ba45bdd3a9951f76.jpg   1024     768   
4    5  wheelchair/images/val/c3542aa1fe056b85.jpg    680    1024   
5    6  wheelchair/images/val/b8ebcca85d76eaa1.jpg   1024     683   
6    7  wheelchair/images/val/d9698162a138414b.jpg    680    1024   
7    8  wheelchair/images/val/a5a1417c00daf0c8.jpg    768    1024   
8    9  wheelchair/images/val/d93b490a89b754c8.jpg    884    1024   
9   10  wheelchair/images/val/b0e1ef2f1d0e0ab1.jpg   1024     561   
10  11  wheelchair/images/val/cc0815f73df3586a.jpg   1024     768   

                 date_captured  license coco_url flickr_url  
0   2021-04-18 13:48:50.878510        1                      
1   2021-04-18 13:48:50.878510        1   

In [147]:
print(f'add #{selected_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} train set')

ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')

add #13 annotations to #7713 train set
train annotations #7726 for train set after augmentation


## WP

In [148]:
index_id_list = find_index_of_images(wp_img_df, image_name_list=val_img_wp)

In [149]:
selected_wp_img_df = create_img_df(wp_img_df, index_id_list)
selected_wp_ann_df = create_ann_df(wp_ann_df, index_id_list)
selected_wp_ann_df = add_area(selected_wp_ann_df)
print(selected_wp_img_df.shape[0])
selected_wp_ann_df.shape[0]

76


2320

In [150]:
print(f'add #{selected_img_df.shape[0]} images to #{aug_img_df.shape[0]} train set')
aug_img_df, selected_wp_ann_df = uodate_id_in_image_ann(selected_wp_img_df, aug_img_df, selected_wp_ann_df)
print(f'val img df after aug shape: {aug_img_df.shape[0]}')


add #11 images to #698 train set
val img df after aug shape: 774


In [151]:
print(f'add #{selected_wp_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} val set')
ann_df_c = concat_ann_df(ann_df_c, selected_wp_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']
print(f'val annotations #{ann_df_c.shape[0]} for train set after augmentation')

add #2320 annotations to #7726 val set
val annotations #10046 for train set after augmentation


### selected_caltech

In [152]:
ca_file = '/home/test_pc/mmlab/mmdetection/datasets/selected_caltech/val.json'

f = open(ca_file)
cl_data = json.load(f)
cl_categories = cl_data['categories']
cl_images = cl_data['images']
cl_annotations = cl_data['annotations']
cl_img_df = pd.DataFrame(cl_images)
cl_ann_df = pd.DataFrame(cl_annotations)
print(cl_img_df.shape[0])
cl_ann_df = add_area(cl_ann_df)

print(cl_ann_df.shape[0])

44
79


In [153]:
print(aug_img_df.shape[0])
selected_img_df = copy.deepcopy(cl_img_df)
selected_ann_df = copy.deepcopy(cl_ann_df)
print(f'add #{selected_img_df.shape[0]} images to #{aug_img_df.shape[0]} val set')
aug_img_df, selected_ann_df = uodate_id_in_image_ann(selected_img_df, aug_img_df, selected_ann_df)

print(f'val img df after aug shape: {aug_img_df.shape[0]}')

774
add #44 images to #774 val set
val img df after aug shape: 818


In [154]:
print(f'add #{selected_ann_df.shape[0]} annotations to #{ann_df_c.shape[0]} val set')
ann_df_c = concat_ann_df(ann_df_c, selected_ann_df)

for idx, row in ann_df_c.iterrows():
    assert idx + 1 == row['id']

print(f'train annotations #{ann_df_c.shape[0]} for train set after augmentation')

add #79 annotations to #10046 val set
train annotations #10125 for train set after augmentation


In [155]:
aug_img_df['date_captured'] = ""
aug_img_df['license'] = ""
aug_img_df['coco_url'] = ""
aug_img_df['flickr_url'] = ""
aug_img_df

,id,file_name,height,width,date_captured,license,coco_url,flickr_url
0,1,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_000294_leftImg8bit.png,1024,2048,,,,
1,2,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_000576_leftImg8bit.png,1024,2048,,,,
2,3,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_001016_leftImg8bit.png,1024,2048,,,,
3,4,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_001236_leftImg8bit.png,1024,2048,,,,
4,5,CityPersons/leftImg8bit_trainvaltest/leftImg8bit/val_all_in_folder/frankfurt_000000_001751_leftImg8bit.png,1024,2048,,,,
...,...,...,...,...,...,...,...,...
813,814,selected_caltech/images/val/set02_V004_412.png,480,640,,,,
814,815,selected_caltech/images/val/set03_V009_1667.png,480,640,,,,
815,816,selected_caltech/images/val/set00_V013_199.png,480,640,,,,
816,817,selected_caltech/images/val/set00_V009_1453.png,480,640,,,,


In [156]:
ann_df_c[ann_df_c.category_id != 1]

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area


In [157]:
ann_df_c

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,0,1.0,"[1839, 382, 31, 71]",1024,2048,2201
1,2,1,1,0,0.0,"[947, 406, 17, 40]",1024,2048,680
2,3,1,1,0,1.0,"[990, 374, 14, 17]",1024,2048,238
3,4,1,1,0,1.0,"[1914, 73, 88, 90]",1024,2048,7920
4,5,1,1,0,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
10120,10121,817,1,0,NaN,"[336, 156, 11, 35]",480,640,385
10121,10122,817,1,0,NaN,"[324, 159, 11, 30]",480,640,330
10122,10123,818,1,0,NaN,"[517, 154, 18, 50]",480,640,900
10123,10124,818,1,0,NaN,"[530, 163, 15, 42]",480,640,630


In [158]:
ann_df_c["iscrowd"].replace({1: True, 0:False}, inplace=True)
ann_df_c['ignore'] = ann_df_c['ignore'].fillna(0)
ann_df_c

,id,image_id,category_id,iscrowd,ignore,bbox,height,width,area
0,1,1,1,False,1.0,"[1839, 382, 31, 71]",1024,2048,2201
1,2,1,1,False,0.0,"[947, 406, 17, 40]",1024,2048,680
2,3,1,1,False,1.0,"[990, 374, 14, 17]",1024,2048,238
3,4,1,1,False,1.0,"[1914, 73, 88, 90]",1024,2048,7920
4,5,1,1,False,1.0,"[1834, 101, 65, 58]",1024,2048,3770
...,...,...,...,...,...,...,...,...,...
10120,10121,817,1,False,0.0,"[336, 156, 11, 35]",480,640,385
10121,10122,817,1,False,0.0,"[324, 159, 11, 30]",480,640,330
10122,10123,818,1,False,0.0,"[517, 154, 18, 50]",480,640,900
10123,10124,818,1,False,0.0,"[530, 163, 15, 42]",480,640,630


In [159]:
print(f"CityPersons: #{aug_img_df[aug_img_df['file_name'].str.contains('CityPersons')].shape[0]}")
# selected_caltech
print(f"selected_caltech: #{aug_img_df[aug_img_df['file_name'].str.contains('selected_caltech')].shape[0]}")
# WiderPerson
print(f"WiderPerson: #{aug_img_df[aug_img_df['file_name'].str.contains('WiderPerson')].shape[0]}")
# wheelchair
print(f"wheelchair: #{aug_img_df[aug_img_df['file_name'].str.contains('wheelchair')].shape[0]}")
# CrowdHuman
print(f"CrowdHuman: #{aug_img_df[aug_img_df['file_name'].str.contains('CrowdHuman')].shape[0]}")
# EuroCity
print(f"EuroCity: #{aug_img_df[aug_img_df['file_name'].str.contains('EuroCity')].shape[0]}")

CityPersons: #425
selected_caltech: #44
WiderPerson: #76
wheelchair: #11
CrowdHuman: #94
EuroCity: #168


### Save json file

In [160]:
aug_data_train = dict()
aug_data_train['info'] = cp_info
aug_data_train['categories'] = cp_data['categories']
aug_data_train['images'] = aug_img_df.to_dict('records')
aug_data_train['annotations'] = ann_df_c.to_dict('records')
with open('/home/test_pc/mmlab/mmdetection/datasets/cp_val.json', 'w') as fp:
    json.dump(aug_data_train, fp)

In [161]:
cp_val = '/home/test_pc/mmlab/mmdetection/datasets/cp_val.json'
f = open(cp_val)
cp_val_data = json.load(f)
# cp_val_licenses = cp_val_data['licenses']
cp_val_cat = cp_val_data['categories']
cp_val_images = cp_val_data['images']
cp_val_annotations = cp_val_data['annotations']
cp_val_img_df = pd.DataFrame(cp_val_images)
cp_val_ann_df = pd.DataFrame(cp_val_annotations)

print(f"CityPersons: #{cp_val_img_df[cp_val_img_df['file_name'].str.contains('CityPersons')].shape[0]}")
# selected_caltech
print(f"selected_caltech: #{cp_val_img_df[cp_val_img_df['file_name'].str.contains('selected_caltech')].shape[0]}")
# WiderPerson
print(f"WiderPerson: #{cp_val_img_df[cp_val_img_df['file_name'].str.contains('WiderPerson')].shape[0]}")
# wheelchair
print(f"wheelchair: #{cp_val_img_df[cp_val_img_df['file_name'].str.contains('wheelchair')].shape[0]}")
# CrowdHuman
print(f"CrowdHuman: #{cp_val_img_df[cp_val_img_df['file_name'].str.contains('CrowdHuman')].shape[0]}")
# EuroCity
print(f"EuroCity: #{cp_val_img_df[cp_val_img_df['file_name'].str.contains('EuroCity')].shape[0]}")

CityPersons: #425
selected_caltech: #44
WiderPerson: #76
wheelchair: #11
CrowdHuman: #94
EuroCity: #168
